In [578]:
import os
import qiskit
from qiskit import transpile, assemble
import torch
from qiskit_aer import AerSimulator
from qiskit import transpile, QuantumCircuit as QiskitQuantumCircuit
from sklearn.metrics import accuracy_score
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Function
from sklearn.metrics import confusion_matrix
import seaborn as sns
import torchxrayvision as xrv
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import ResNet18_Weights
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from torchvision.io import read_image

from sklearn.model_selection import train_test_split

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os, random, json, cv2, math
from PIL import Image
from tqdm import tqdm

import qiskit
from qiskit import transpile, assemble

In [579]:
QC_outputs = ['000', '001', '010', '011', '100', '101', '110', '111']

In [580]:
class MammoDataset(Dataset):
    def __init__(self, root_dir, train=True):
        self.image_paths = []
        self.labels = []
        
        if train:
            self.transform = transforms.Compose([transforms.Resize((250, 250)),
                                 transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        else: 
            self.transform = transforms.Compose([transforms.Resize((250, 250)),
                                 transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
  
        for label in ['0', '1']:
            folder_path = os.path.join(root_dir, label)
            for img_name in os.listdir(folder_path):
                self.image_paths.append(os.path.join(folder_path, img_name))
                self.labels.append(int(label))

    def __len__(self):
        return len(self.labels)
    
    def get_transform(data_augmentation=False):
        if data_augmentation:
            transform = transforms.Compose([transforms.Resize((250, 250)),
                                        transforms.RandomRotation(10)])

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        #image = torch.Tensor(np.array(image))
        if self.transform:
            image = self.transform(image)

        # Converte a imagem para um tensor do PyTorch
        #image = torch.tensor(image, dtype=torch.float32)

        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return image, label

In [581]:
# class DenseNetNN(nn.Module):
#     def __init__(self, *args, **kwargs):
#         super(DenseNetNN, self).__init__()
#         self.model = xrv.models.DenseNet(weights="all").features
#         #self.embedding_layer = nn.Linear(1024, 128)

#     def forward(self, x):
#         x1 = self.model(x)

#         return x1

In [582]:
class Resnet(torch.nn.Module):
    def __init__(self, out_features = 2):
        super(Resnet, self).__init__()
        self.resnet = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.resnet.fc = torch.nn.Linear(self.resnet.fc.in_features, out_features)
    def forward(self, x):
        return self.resnet(x)

In [583]:
class QuantumCircuit:
    """ 
    This class provides a simple interface for interaction 
    with the quantum circuit 
    """
    
    def __init__(self, n_qubits, backend, shots):
        # --- Circuit definition ---
        self._circuit = qiskit.QuantumCircuit(n_qubits)
        
        all_qubits = [i for i in range(n_qubits)]
        self.theta_0 = qiskit.circuit.Parameter('theta0')
        
        self._circuit.h(all_qubits)
        self._circuit.barrier()
        self._circuit.ry(self.theta_0, all_qubits)
        #self._circuit.rx(self.theta, all_qubits)
        
        self._circuit.measure_all()
        # ---------------------------

        self.backend = backend
        self.shots = shots
    
    def run(self, thetas):
        param_values = {
            self.theta_0: thetas[0],
        }
        

        bound_circuit = self._circuit.assign_parameters(param_values)

        transpiled_circuit = transpile(bound_circuit, self.backend)

        job = self.backend.run(transpiled_circuit, shots=self.shots)
        
        result = job.result().get_counts()
        
        expectations = []
        if type(result)==list:
            for i in result:
                counts = np.array(list(i.values()))
                states = np.array(list(i.keys())).astype(float)
            
                # Compute probabilities for each state
                probabilities = counts / self.shots
                # Get state expectation
                expectation = np.sum(states * probabilities)

                expectations.append(expectation)
        else:
            counts = np.array(list(result.values()))
            states = np.array(list(result.keys())).astype(float)
        
            # Compute probabilities for each state
            probabilities = counts / self.shots
            # Get state expectation
            expectation = np.sum(states * probabilities)

            expectations.append(expectation)
        return np.array(expectations)

In [584]:
class HybridFunction(torch.autograd.Function):
    """ Hybrid quantum - classical function definition """
    
    @staticmethod
    def forward(ctx, input, quantum_circuit, shift):
        """ Forward pass computation """
        ctx.shift = shift
        ctx.quantum_circuit = quantum_circuit
        expectation_z = ctx.quantum_circuit.run(input.tolist()[0])
        result = torch.tensor([expectation_z])
        
        ctx.save_for_backward(input, result)

        return result
        
    @staticmethod
    def backward(ctx, grad_output):
        """ Backward pass computation """
        input, expectation_z = ctx.saved_tensors
        input_list = np.array(input.tolist()[0])
        
        shift_right = input_list + np.ones(input_list.shape) * ctx.shift
        shift_left = input_list - np.ones(input_list.shape) * ctx.shift
        gradients = []
        for i in range(len(input_list)):
            expectation_right = ctx.quantum_circuit.run([shift_right[i]])
            expectation_left  = ctx.quantum_circuit.run([shift_left[i]])
            gradient = expectation_right - expectation_left
            gradients.append(gradient)
        
        gradients = np.array([gradients]).T
        return torch.tensor([gradients]).float() * grad_output.float(), None, None



class Hybrid(nn.Module):
    """ Hybrid quantum - classical layer definition """
    
    def __init__(self, n_qubits, backend, shots, shift):
        super(Hybrid, self).__init__()
        self.quantum_circuit = QuantumCircuit(n_qubits, backend, shots)
        self.shift = shift
        
    def forward(self, input):
        batch_results = []
        for i in range(input.shape[0]):
            result = HybridFunction.apply(input[i].unsqueeze(0), self.quantum_circuit, self.shift)
            batch_results.append(result)
        return torch.cat(batch_results)

In [585]:
class qcNet(nn.Module):
    def __init__(self):
        super(qcNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, stride=2, padding=1)
        self.conv2 = nn.Conv2d(6, 15, kernel_size=3, stride=2, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)
        self.drop1 = nn.Dropout2d(p=0.2)
        self.drop2 = nn.Dropout2d(p=0.5)
        self.fc1 = nn.Linear(55815, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)
        sim = AerSimulator()
        self.hybrid = Hybrid(self.fc3.out_features, sim, 1000, np.pi / 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = self.drop1(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.drop1(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.drop2(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.hybrid(x)
        x = torch.cat((x, 1 - x), -1)
        return x

In [586]:
train_dir = 'C:\\Users\\Win10\\PycharmProjects\\QuantumResearch\\INbreast\\data\\png\\train'
test_dir = 'C:\\Users\\Win10\\PycharmProjects\\QuantumResearch\\INBreast\\data\\png\\test'

# Instancia os datasets
train_dataset = MammoDataset(root_dir=train_dir)
test_dataset = MammoDataset(root_dir=test_dir, train=False)

# Cria os DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [587]:
model = qcNet().cuda()
loss_func = nn.NLLLoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs = 3
loss_list = []
thetas = []

In [588]:
model.train()
for epoch in range(epochs):
    total_loss = []
    
    for batch_idx, (image, label) in enumerate(train_loader):
        label = label.unsqueeze(1)
        
        # Converta os tensores para o tipo correto
        image = image.cuda()
        label = label.cuda()

        optimizer.zero_grad()  # Zere os gradientes

        # Passe as imagens pelo modelo
        prediction = model(image)

        prediction = prediction.cuda()

        #Get prediction label from the index with the maximum score
        #prediction_label = torch.argmax(prediction, dim=1)

        #prediction_label = prediction_label.unsqueeze(1)

        print(label, prediction)
        # Calcule a perda
        loss = loss_func(prediction, label.squeeze())

        # Propagação para trás e atualização dos pesos
        loss.backward()
        optimizer.step()

        # Armazene as informações
        total_loss.append(loss.item())
        print(f"  Batch {batch_idx + 1} concluído com loss: {loss.item():.4f}")
        
    avg_loss = np.mean(total_loss)
    loss_list.append(avg_loss)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss}")

    model_save_path = f"model_epoch_{epoch}.pth"
    torch.save(model.state_dict(), model_save_path)

tensor([[0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.]], device='cuda:0') tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')


RuntimeError: "nll_loss_forward_reduce_cuda_kernel_1d" not implemented for 'Long'

In [ ]:
model.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for image, label in test_loader:
        image, label = image.type(dtype=torch.cuda.FloatTensor), label.type(dtype=torch.cuda.LongTensor)
        try:
            prediction = model(image)
            predicted_class = prediction.argmax(dim=1)
            all_labels.extend(label.cpu().numpy())
            all_predictions.extend(predicted_class.cpu().numpy())
        except:
            continue
        print(label, predicted_class)

# Calcular acurácia
accuracy = accuracy_score(all_labels, all_predictions)
print(f"Acurácia: {accuracy:.4f}")

In [ ]:
# Gerar matriz de confusão
conf_matrix = confusion_matrix(all_labels, all_predictions)

# Plotar matriz de confusão
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Negativo', 'Positivo'],
            yticklabels=['Negativo', 'Positivo'])
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.show()